In [7]:
import langchain
print(langchain.__version__)

1.2.0


In [8]:
from langchain_openai import ChatOpenAI

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hello!")
]

print("LangChain is working correctly!")
print(f"Created {len(messages)} messages")

LangChain is working correctly!
Created 2 messages


LangChain a été divisé en plusieurs morceaux pour être plus léger :

langchain-core : Contient les schémas de base (messages, documents).

langchain-community : Contient les intégrations tierces.

langchain : Contient les chaînes et la logique de haut niveau.

Une maintenance adéquate est essentielle pour garantir un environnement stable et efficace après une installation réussie de LangChain. Si l'installation constitue le point de départ, le maintien de la stabilité implique une gestion rigoureuse des dépendances et un contrôle rigoureux des versions. L'alignement des packages principaux et optionnels permet d'éviter les conflits potentiels.

In [10]:
import langchain
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import BaseOutputParser 

# 1. Définition du Parser
class SimpleParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        # On s'assure que text est bien une chaîne
        return str(text).strip().upper()

# 2. Définition du Prompt
template = "Transform this text: {input_text}"
prompt = PromptTemplate.from_template(template)
parser = SimpleParser()

# 3. La Chaîne (Modifiée pour le test)
# Pour que ça marche SANS modèle, on doit ajouter une étape qui extrait le texte du prompt
chain = prompt | (lambda x: x.to_string()) | parser 

# 4. Exécution
result = chain.invoke({"input_text": "hello world"})

print(f"LangChain version: {langchain.__version__}")
print(f"Chain result: {result}")

LangChain version: 1.2.0
Chain result: TRANSFORM THIS TEXT: HELLO WORLD


In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"API key loaded: {api_key[:8]}...")
else:
    print("No API key found - check your .env file")

No API key found - check your .env file


- se rensigner : Pour la gestion des documents, ajoutez des outils tels que pypdf pour les PDF et beautifulsoup4 pour le scraping Web.

- Ensuite, choisissez une base de données vectorielle. Pour les tests locaux, Chroma Il s'agit d'une option simple nécessitant une configuration minimale. Pour une production à plus grande échelle, privilégiez les bases de données offrant des performances supérieures, même si elles peuvent nécessiter des configurations d'API supplémentaires.

- Vous aurez également besoin de clés API pour activer les fonctionnalités clés. Obtenez une clé API OpenAI pour accéder aux intégrations telles que text-embedding-ada-002 et des modèles tels que gpt-4 or gpt-3.5-turbo. Stockez ces clés en toute sécurité à l'aide de variables d'environnement ou d'outils tels que AWS Secrets Manager.
 
- Commencez par sélectionner les outils appropriés pour charger vos documents. Par exemple, PyPDFLoader gère les fichiers PDF tout en conservant leur formatage, et WebBaseLoader peut extraire le contenu des sites Web avec des options d'analyse flexibles.

- Une fois chargé, divisez le texte en morceaux gérables pour améliorer la précision de la récupération. RecursiveCharacterTextSplitter est un outil polyvalent, offrant un équilibre entre la taille des blocs et leur chevauchement. Par exemple, des blocs plus petits de 500 à 800 caractères conviennent parfaitement aux FAQ, tandis que des blocs plus grands de 1,500 2,000 à XNUMX XNUMX caractères sont plus adaptés aux documents techniques. Une fois le texte préparé, vous pouvez passer à la génération d’intégrations.




In [12]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("document.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["", "", " ", ""]
)
splits = text_splitter.split_documents(documents)

ModuleNotFoundError: No module named 'langchain.document_loaders'

Les incorporations convertissent des fragments de texte en représentations numériques qui capturent leur signification. text-embedding-ada-002 Le modèle est un choix fiable, générant des vecteurs de 1,536 XNUMX dimensions adaptés à divers contenus.

In [ ]:
''' 
j'aimerais bien le faire avec pinecone :
'''
Voici comment générer et stocker des intégrations à l'aide de Chroma :

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

Le processus de recherche identifie les fragments de documents les plus pertinents pour une requête. L'utilisation d'un outil de recherche par similarité avec k=4 récupère les quatre premiers morceaux, en équilibrant les détails et les limites d'entrée pour le modèle de langage.

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

L'ingénierie des invites est un autre aspect essentiel. Une invite bien conçue garantit que le modèle de langage utilise efficacement le contexte récupéré. Par exemple :

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the question based on the provided context. If the context doesn't contain relevant information, say so clearly.

Context: {context}

Question: {question}

Answer:
""")

Pour les besoins avancés, des techniques telles que la récupération multi-requêtes ou les méthodes hybrides (combinant similarité sémantique et correspondance de mots-clés) peuvent améliorer les résultats, en particulier pour le contenu technique.



Construire le système RAG complet
L'étape finale consiste à intégrer tous les composants dans un système RAG unifié. create_retrieval_chain La fonction simplifie cela en coordonnant la récupération et la génération.

Voici un exemple :

Un de mes problemes est les clé API payante, comment faire ? Je crois que avec mistral il y a des requetes gratuites 

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "Your question here"})
print(response["answer"])

INTERET N8N : L'amélioration des performances de LangChain RAG (Retrieval-Augmented Generation) implique d'affiner les paramètres de recherche et les méthodes de recherche afin de garantir des réponses précises et contextuelles. L'utilisation de techniques de recherche intelligentes et l'optimisation de la configuration des blocs permettent d'améliorer considérablement l'efficacité du système.

Combiner des méthodes de recherche, telles que les approches sémantiques et par mots-clés, peut améliorer la précision dans des domaines spécialisés. Voici un exemple de configuration d'un outil de recherche hybride :

from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(splits)
ensemble_retriever = EnsembleRetriever(
    retrievers=[vectorstore.as_retriever(), bm25_retriever],
    weights=[0.6, 0.4]
)

De plus, les techniques d’expansion de requêtes, comme la récupération multi-requêtes, peuvent générer une formulation alternative pour capturer un contexte plus large et réduire l’impact des requêtes mal formulées.


KEASKO LA TEMPERTATURE ??? : Les réglages de température jouent également un rôle essentiel dans la qualité du rendu. Pour les tâches factuelles, des valeurs de température plus basses permettent de minimiser les hallucinations, tandis que des valeurs légèrement plus élevées sont plus adaptées aux tâches exigeant créativité ou flexibilité.


Déploiement de production


- Pour améliorer l'efficacité, implémentez des couches de mise en cache pour les documents fréquemment consultés. Des outils comme Redis or Memcached Vous pouvez stocker les résultats d'intégration pour les requêtes courantes, réduisant ainsi la charge de vos services d'intégration. Définissez des valeurs de durée de vie (TTL) selon que vos données sont statiques ou fréquemment mises à jour.

- Pour les applications à fort trafic, répartissez la charge sur plusieurs points de terminaison d'API d'intégration afin d'éviter toute limitation du débit. Vous pouvez également envisager d'utiliser des modèles d'intégration locaux pour maintenir des performances constantes en cas de forte demande
  
-


Sécurité et conformité des données

- Le contrôle d'accès dans les systèmes RAG peut être complexe, car les utilisateurs accèdent indirectement aux informations via les réponses de l'IA. Mettez en œuvre des autorisations au niveau des documents en étiquetant les fragments de documents avec des métadonnées et en filtrant les résultats de récupération en fonction des rôles des utilisateurs avant le traitement. ( pas forcement besoin ici car veille strategique )
 

- Les politiques de conservation des données doivent prendre en compte à la fois les documents sources et les intégrations générées. Des réglementations comme le RGPD peuvent exiger des mécanismes de suppression de données utilisateur spécifiques des bases de données vectorielles. Prévoyez donc la suppression complète des données dès le départ.

- Les journaux d'audit sont essentiels à la conformité et à la sécurité. Ils doivent enregistrer des informations clés telles que les identifiants des utilisateurs, les horodatages, les modèles de requête, les documents récupérés et les réponses générées. Assurez-vous de minimiser l'exposition des données sensibles tout en conservant suffisamment de détails pour les rapports de conformité et la détection des fuites de données potentielles.

Étant donné que votre Mac est un modèle de 2012, nous allons coder de manière à ce qu'il ne sature pas sa mémoire vive (8 Go) en traitant les documents par petits morceaux.